# Quantum Machine Learning challenge

Welcome to the Quantum Machine Learning challenge of the MQST Qiskit Fall Fest!
In this challenge, you will explore how quantum computing (QC) and machine learning (ML) can be brought together in the exciting field of Quantum Machine learning (QML). You will start using quantum computers for a simple classical classification task, and your fundamental challenge will be to design a "Quantum Neural Network" (QNN) that classifies input quantum data according to the phase it corresponds to.

#### First install the necessary packages:

In [ ]:
# Uncomment to install the necessary packages

%pip install --upgrade pip
%pip install numpy==2.3.4
%pip install scipy==1.16.2
%pip install matplotlib==3.10.7
%pip install jupyter
%pip install qiskit[visualization]==2.2
# # For Local Linux or Mac, you can use the following command:
# # %pip install 'qiskit[visualization]'
%pip install qiskit_algorithms==0.4.0
%pip install qiskit_aer==0.17.2
%pip install qiskit_ibm_runtime==0.41.1
%pip install qiskit_ibm_transpiler==0.14.3

---

## 1.Introduction

If not of machine learning, everyone has heard of artificial intelligence. From humbler origins, to the current large language models, machine learning has permeated everything and become widespread. We will focus on the supervised learning paradigm. Simplifying, we look to solve a problem where we want to assign to each input $x$ some corresponding output $y = f(x)$. In fact this is a very general way to see any problem, so we note two distinct examples:
- An example is finding how many prime factors an integer has. This is a simple problem in the sense that we **know** how to express the $f$ that solves it, although that does not mean it is easy to do so efficiently. In this case, it has only been after Shor's quantum algorithm that this problem, previously thought of as computationally hard, has been "broken".
- A much different example is the task of classifying pictures of cats and dogs. In this case we know, or rather we assume, that there is some function that solves the problem because **we** can solve it. We can manually look at pictures and tell if it contains a cat or a dog. In spite of this, no one is able to come up with a simple and direct computer algorithm to solve this task.

It is for these latter kind of problems where supervised learning shines the most. The fundamental principle is that instead of trying to find this goal function $f$ directly, we take a generic parametrized guess. Now, exploiting that we can previously prepare some sample inputs/outputs (such as prelabelled cats and dogs pictures), we tweak our parameters to find the best fit. In fact, you have already done supervised learning before, as taking a linear regression is technically just that, where the guess (ansatz) is a linear function. Similarly, just as a linear regression will not directly work for a non linear problem, this choice of our ansatz is important for each problem.

On the other hand, quantum computing has emerged to find the physical limit of information processing, and ideas of combining it with machine learning happened soon after. There are many ways one can quantumly enhance machine learning, not just for show by putting the "quantum" in front. We will focus on using a parametrized quantum computer as ansatz for the problem solving function $f$, a so-called (abusing language) Quantum Neural Network. Your goal during this whole challenge is to design good models for quantum machine learning, which is still an ongoing research question while quantum resources are limited to perform big scale tests.

First you will apply it for a classical problem, which will probably turn out bad! Even if Quantum Computers can do some tasks better, it is important to realise this is not an actually easy goal to achieve. Quantum intuition can differ a lot from classical intuition, so it is hard to come up with good designs. Finally, you will apply it for Quantum Phase Recongition, where the quantum nature of the problem will be a much better fit.


### A simple task

Consider the following classical data points, which are classified into one of two classes. Our goal is to learn to perform this classification, not by memorizing every point but rather by learning the underlying rule, so we will able to classify new points too.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Set random seed so results are repeatable
np.random.seed(2025)

## Underlying classification, outside of square or inside
def underlying_function(R: np.ndarray) -> float:
    return np.max(np.abs(R), axis=1) > 1/np.sqrt(2)


## Prepare random dataset
Nsamples = 500
R = 2*np.random.random((Nsamples,2)) - np.array([1,1])  # Data points, uniform in [-1,1]x[-1,1]

y = underlying_function(R)   # Classification

R_class1 = R[y]                    # Class +1: Outside of square
R_class0 = R[np.logical_not( y )]  # Class  0: Inside  of square

plt.axis('equal')
plt.scatter(R_class1[:,0], R_class1[:,1], label="Class  1")
plt.scatter(R_class0[:,0], R_class0[:,1], label="Class  0")
plt.legend(loc = "upper left")

The underlying function is simple: points inside the $\sqrt{2}$ square are in class 0 and points outside in class 1, but we must now try to learn this in a generic way, even if this is a simple case. We start by splitting our data into a training set and a testing set. The training set will be the points we will use to tweak our model so it classifies them correctly, while the testing points will be used to test if the model generalises well.

In [ ]:
# We split data into training set (to optimize directly) and test set (to check other points also get classified correctly)
train_size = 0.5
Ntrain = int( Nsamples * train_size )

train_R, test_R = R[:Ntrain], R[Ntrain:]
train_y, test_y = y[:Ntrain], y[Ntrain:]

### 1.1 Classical to Quantum

We will learn to classify points, by learning the underlying distribution, which you can see is $f(\vec{x}) = -1$, if $\vec{x} \in [-\frac{\sqrt{2}}{2}, \frac{\sqrt{2}}{2}] \times [-\frac{\sqrt{2}}{2}, \frac{\sqrt{2}}{2}] $; $\quad-1$ otherwise. To do this, you will now create a Quantum Neural Network (QNN) and later you will train it on the data.

Let's create the Quantum Neural Network, the first step is to encode your problem, your data, into the quantum computer.
> Hint: As an example you may use Qiskit's `qiskit.circuit.library.ZZFeatureMap` or `ZFeatureMap` to encode real numbers as angles of rotation on each qubit

You will also want to define some processing layers that use trainable parameters.
> Hint: If you do not want to manually construct them, with gates, you may use provided examples like `qiskit.circuit.library.EfficientSU2` or `qiskit.circuit.library.TwoLocal`

The principle is to use gates, such as $R_Y$ rotations, where the angles will be given according to some input/tunable parameters


**The code snippets are orientative to guide you through the problems, but feel free to come up with your own solutions**. For instance, you may mix input parameters and trainable parameters on the same gate, or intertwine gates (also called data re-reuploading) instead of applying encoding first and processing later.


In [ ]:
## Your code here
import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap     # Example featuremap/encoding
from qiskit.circuit.library import EfficientSU2, RealAmplitudes  # Example processing layer

# Example of parameters you may use. Default input. If using default, inputs are called x and trainable_parameters θ
inputs = ParameterVector('x', 2)
trainable_parameters = ParameterVector('θ', 16)

# By default, you may use only 2 qubits, one for each coordinate

def encoding(inputs: ParameterVector) -> QuantumCircuit:
    """Returns an encoder that will encode a point in space (x,y) into a quantum circuit.

    Args:
        inputs (ParameterVector): parameters corresponding to input (x, y)

    Returns:
        encoding_circuit (QuantumCircuit): Returns a QuantumCircuit, which encodes the classical data into the circuit
    """
    # YOUR CODE HERE

    return encoding_circuit


def processing(trainable_parameters: ParameterVector) -> QuantumCircuit:
    """Returns a processing layer that will be trained to try to solve our problem

    Args:
        trainable_parameters (ParameterVector): parameters that will be trained to attempt to solve the problem

    Returns:
        processing_circuit (QuantumCircuit): Returns a QuantumCircuit that encodes the data into the circuit
    """
    # YOUR CODE HERE

    return processing_circuit


Here you may draw your encoding,

In [ ]:
encoding_circuit = encoding(inputs)
encoding_circuit.decompose().draw(output='mpl', style='textbook')

processing

In [ ]:
processing_circuit = processing(trainable_parameters)
processing_circuit.decompose().draw(output='mpl', style='textbook')

and their combination:
> Again, remember you can join encoding and processing layers differently, or mix them

In [ ]:
# Assign our parameters, may need to adapt
encoding_circuit.assign_parameters(inputs, inplace=True)
processing_circuit.assign_parameters(trainable_parameters, inplace=True)

# Define our QNN Circuit
qnn_circuit = encoding_circuit.compose(processing_circuit)
qnn_circuit.draw(output='mpl', style='textbook')


### 1.2. Finish your QNN

You have prepared a quantum circuit, parametrized, that will attempt to classify our data, you now need to define how to interpret the circuit outputs to do the classification. We can do so by reading the output, for instance the first qubit, and using and assigning the class according to the result (measuring 0 or measuring 1)


Keep in mind that Qiskit uses Little Endian notation (top qubit q[0] is last qubit and so on $|abc\rangle \to$ q[0]=c, q[1]=b, q[2]=a), and that the default sampler output is the integer representation of the bitstring you need to interpret.

To do this, define a forward function, to evaluate running our QNN, and also a way to compute gradients. Use the `ParameterShift` method, that qiskit already provides. The important idea is that it is hardware compatible. If you have trouble with implementing gradients, you may focus on other aspects (and use gradient-free optimizers like SPSA)

> Again, all function suggestions are orientative. If you prefer you may also work with Estimators (expectation values) instead of Samplers.

In [ ]:
#from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN    # Libraries which could simplify process, but icompatible with qiskit 2>
#from qiskit_machine_learning.algorithms import NeuralNetworkClassifier
from qiskit_ibm_runtime import Sampler
from qiskit_aer import AerSimulator
from qiskit_algorithms.gradients import ParamShiftSamplerGradient


def interpreter(bit_int: int) -> int:
    """Takes the all qubit measurement integer output of the quantum circuit and reads only the last qubit (top in diagram), if necessary. Can also be done more carefully by using qiskit measurements QuantumCircuit.measure(...)
    Args:
        bit_int (int): bit_measurement of all qubits in integer form, values in {0, 1, ..., 2**Nqubits-1}
    Returns:
        class_int (int): classification, 0: Class 0, 1: Class +1. Corresponding to last qubit
    """
    # YOUR CODE HERE
    return class_int

output_shape = 2 # if we read only one bit



# Define a forward pass of your QNN
def forward(qnn_circuit: QuantumCircuit, inputs: np.ndarray, trainable_parameters: np.ndarray, sampler: Sampler) -> np.ndarray:
    """
    "Forward pass of the Q neural network", computes output of

    Args:
        qnn_circuit (QuantumCircuit): circuit to run, including some input encoding and trainable processing gates.
        inputs (np.ndarray): data inputs encoded.
        trainable_parameters (np.ndarray): neural network ansatz parameters.
        sampler (Sampler): Sampler to use.

    Returns:
        qnn_output (np.ndarray): Array for the probability distribution or samples of measurement outcomes of circuit
    """
    # YOUR CODE HERE

    return qnn_output



# Define a forward pass of your QNN
def backwards(qnn_circuit: QuantumCircuit, inputs: np.ndarray, trainable_parameters: np.ndarray, sampler: Sampler) -> np.ndarray:
    """
    "Backwards pass of the Q neural network", compute gradients of parameters using ParamShiftSamplerGradient.

    Args:
        qnn_circuit (QuantumCircuit): circuit to run, including some input encoding and trainable processing gates.
        inputs (np.ndarray): data inputs encoded.
        trainable_parameters (np.ndarray): neural network ansatz parameters.
        sampler (Sampler): Sampler to use.

    Returns:
        qnn_output (np.ndarray): Array for the probability distribution or samples of measurement outcomes of circuit
    """
    # YOUR CODE HERE

    return qnn_output



### 1.3 Train your QNN

Now you must train your QNN, to do this define a loss funciton and an optimizer.


By default, use a mean squared error loss function
$$\sum (QNN_{\vec{\theta}}(\vec{x_i})-y_i)^2,$$
where $QNN()$ represents the output of your quantum circuit (forward pass).  You can use scipy minimizers, in particular start using the COBYLA optimizer first, which is gradient free, from and then use the L_BFGS_B, which is slower but uses gradients.

Store also the loss evolution (and other parameters you consider interesting) to plot them later.

> Hint: Remember to look for Qiskit resources, for instance you may find [this](https://quantum.cloud.ibm.com/learning/en/courses/quantum-machine-learning/qvc-qnn) tutorial useful





In [ ]:
# Define some loss functions for your training, as an example here you have an MSE

def loss_function(prediction: float, label: float):
    """Example MSE loss term for a single input"""
    return np.mean((prediction - label)**2)

In [ ]:
from scipy.optimize import minimize

# Define loss value_list that will be updated
loss_value_list = []

# Your code here


In [ ]:
qc = QuantumCircuit(3)
from qiskit.circuit.library import StatePreparation

s0 = np.array([0]*8)
s0[1] = np.sqrt(1/2)
s0[3] = np.sqrt(1/2)


StatePreparation()

#### 1.4 Plot your reconstructed classification

In [ ]:
def plot_prediction(QNN_classifier):
    """Auxiliar function to plot classification. Feel free to modify it"""
    # Making prediction
    prediction = QNN_classifier.predict(R)

    is_one = prediction == 1

    predicted_R_class1 = R[is_one]                    # Class +1: Outside of square
    predicted_R_class0 = R[np.logical_not( is_one )]  # Class  0: Inside  of square

    ## PLOT Prediction
    plt.axis('equal')

    # Draw square
    plt.plot([-np.sqrt(2)/2, -np.sqrt(2)/2, np.sqrt(2)/2, np.sqrt(2)/2, -np.sqrt(2)/2], [-np.sqrt(2)/2, np.sqrt(2)/2, np.sqrt(2)/2, -np.sqrt(2)/2, -np.sqrt(2)/2], 'k--')
    plt.scatter(predicted_R_class1[:,0], predicted_R_class1[:,1], label="Class  1")
    plt.scatter(predicted_R_class0[:,0], predicted_R_class0[:,1], label="Class  0")
    plt.legend(loc = "upper left")

plot_prediction(QNN_classifier)

### 1.5 Explore

#### 1.5.1 Optimizers

You can repeat the previous steps using different optimizers, in particular try again with L-BFGS and try to tweak the circuits

In [ ]:
# Code here

#### 1.5.2 Data re-uploading

Instead of just changing the encoding layer and the processing layer, explore here if you can obtain better results by intertwining them:

Build your circuit with an encoding layer, a processing layer, another encoding layer and another processing layer. As this will increase the complexity, you may also want to simplify the layers themselves.

In [ ]:
# Code here

## 2. Putting more quantum in Quantum Machine Learning

Your task will be to train a quantum machine learning model to classify which phase of matter corresponds to each input quantum state.

We are going to study now the XY model on a periodic chain, defined as

$$H = -\sum^N_{j=1} \left [ \left(\frac{1+\gamma}{2}\right) \sigma^x_j \sigma^x_{j+1} + \left(\frac{1-\gamma}{2}\right) \sigma^y_j \sigma^y_{j+1}  + h \sigma^z_j \right],$$

with $\sigma_{N+1}=\sigma$.
It has the following simple phase diagram, where we will focus on the $h,\gamma>0$ region, with phase transitions at $h = 1$ and at the circular boundary $\gamma^2 + h^2 = 1$:

![image.png](attachment:image.png)

### 2.1 Implement the hamiltonian


First you must implement the hamiltonian in Qiskit

In [ ]:
from qiskit.quantum_info import SparsePauliOp
# Code here

def XY_hamiltonian(gamma: float, h: float, Nsites: int) -> SparsePauliOp:
    """Returns the XY hamiltonian

    Args:
        gamma (double): gamma parameter of hamiltonian
        h (double): h transverse field parameter
        Nsites (int): Number of spin sites, equivalent to number of qubits for a one to one mapping

    Returns:
        hamiltonian (SparsePauliOp): XY_hamiltonian with the given parameters
    """
    # YOUR CODE HERE
    pass

### 2.2 Solve it classically

Create a classical method to find the ground states of the hamiltonian
> hint: use libraries such as eigsh to diagonalise the hamiltonian with `.to_matrix()`

The steps are similar to the case before, but now the inputs do not need to be directly embeded into the quantum computer as they are already quantum states. You can use `.prepare_state()`. You can start simple, with N=6 qubits.

In [ ]:
def XY_ground_state(gamma: float, h: float, Nsites: int) -> np.ndarray:
    """Returns the ground state for the corresponding XY hamiltonian

    Args:
        gamma (double): gamma parameter of hamiltonian
        h (double): h transverse field parameter
        Nsites (int): Number of spin sites, equivalent to number of qubits for a one to one mapping

    Returns:
        ground_state (np.ndarray): ground state of the corresponding hamiltonain
    """
    # YOUR CODE HERE
    pass

### 2.3 Construct the QNN

In this case, you won't need to encode your data, as you can map each spin to a qubit directly. You can input states into qiskit circuits directly, and this is how you can deal with this challenge, with tools like StatePreparation. Construct your QNN here, you can adapt your processing layers you designed in section 1.2

In [ ]:
from qiskit.circuit import Parameter
from qiskit.primitives import BaseSamplerV2
from qiskit_ibm_runtime import Sampler  # actual to use

def QNN_XY_circuit(parameters: np.ndarray) -> QuantumCircuit:
    """Returns a Quantum neural Network model that will attempt to solve the phase classification task after trayining.
    Args:
        gamma (double): gamma parameter of hamiltonian

    Returns:
        QNN (np.ndarray): Quantum Neural Network that will proces your states
    """
    # YOUR CODE HERE
    pass


Define as in the previous sections some forward passes and backwards passes to compute gradients and you may reuse the loss function

In [ ]:
# Your code here

### 2.4 Train it for your quantum data

Use your classical solver to find the ground states for your system at different parameters, and use that to train your model to classify states. Loop over any random points h and gamma you want and define previously the loss functions and gradients to optimize your QNN.

 Finally, draw the phase diagram that your system detects, defining also a prediciton function

In [ ]:
# Your code here

In [ ]:
def draw_predicted_phasediagram(predict_XY) -> None:
    """Plots the predicted phase diagram.
    """
    prediction = []

    gamma_range = np.linspace(0,2)
    h_range = np.linspace(0,2)
    for gamma in gamma_range:
        prediction.append([])
        for h in h_range:
            prediction[-1].append(predict_XY(gamma, h))

    gamma_mesh, h_mesh = np.meshgrid(gamma_range, h_range)
    plt.contourf(gamma_mesh, h_mesh, prediction)
    plt.show()

draw_predicted_phasediagram(predict_XY)

# Your code here

### 2.5 Analysis

Repeat the previous steps using different optimizers: SPSA/COBYLA/BFGS/ADAM and gate ansatz.

> Hint: As some criterion, you can use gates related to the elements in the hamiltonian. For instance $R_X$, $R_{XX}$ gates etc. You may also consider some other properties of your system, such as symmetries.

In [ ]:
# Your code here

What kind of model works best, and optimizer?

### 2.6 Constrained training

Consider that using as training data many points across the whole phase diagram is very ideal, in general you may be constrained to some easier subspaces to reproduce/generate.

Perform the previous steps, training and predicting the phase diagram, but with constrained initial data points. Compare different cases:

Take points across the line $gamma=1$, across the line $h = 0.5$, and across the horizontal and vertical lines crossing at $\gamma, h = 0$

In [ ]:
# Your code here

How do the results change with the training region?

### 3. More questions

### Scalability
The fundamental principle that motivates quantum machine learning is the search for some quantum advantage with respect to classical machine learning. To simulate a generic quantum computer requires exponential time, although smart techniques such as tensor networks can dequantize simple or very structured circuits, so the ideal criterion is to have our models require at most polynomial resources with the number of qubits. You are invited to tackle this in any way you want, where the general idea is to scale the system for different sizes and see how it fares. Next we provide an example criterion that has become standard, the barren plateau phenomena.

It turns out that the expressability of quantum circuits, due to the exponential scaling of the Hilbert space, comes at a cost, many problems have gradients vanishing exponentially with number of qubits:

**Compute the variance of gradients of your loss functions for random initial parameters of your circuits, and plot how these variances scale with increasing number of sites/qubits**


In [ ]:
# Your code here

### Unsupervised learning
Instead of learning to classify states, having been provided with the knowledge of the existing phases, try now to discover which phases there are. That is, make an unsupervised quantum machine learning paradigm to identify the existence of the different phases the model has.
[This paper](https://arxiv.org/abs/2402.11022) presents an approach to do this. An idea is that instead of performing a classifcation itself, and defining the loss by penalising cases where it differs from the known data, you should compress the quantum states into a smaller subsystem, training it at some point. The success of this compression, after training, which will be the output of your QNN, will be better for states similar to the initial state, and through this you may find

In [ ]:
# Your code here

### Noise and sampling

Introduce sampling and noise into the mix! Do the previous results hold? Do some optimizers or architechtures become better now?
 You could try to also use IBM hardware, but it is not recommended because the state preparation, from arbitary statevectors, is not very hardware friendly.

In [ ]:
# Your code here

## References
[1]. Challenges and Opportunities in Quantum Machine Learning

[2]. Noisy intermediate-scale quantum (NISQ) algorithms

[4] Franchini, Fabio. An introduction to integrable techniques for one-dimensional quantum systems. Vol. 940. Cham: Springer International Publishing, 2017. ([url](https://arxiv.org/abs/1609.02100))